In [1]:
import os
import glob
import numpy as np
from tensorflow.keras import layers
from tensorflow import keras 
import tensorflow as tf
import cv2
from PIL import Image

In [1]:
# get location of python
import sys
print(sys.executable)

c:\Users\texas\.pyenv\pyenv-win\versions\3.8.10\python.exe


In [2]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [3]:
from datasets import load_dataset # Hugging face datasets 

dataset = load_dataset("kmewhort/quickdraw-bins-1pct-sample")

c:\Users\texas\.pyenv\pyenv-win\versions\3.8.10\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset parquet (C:/Users/texas/.cache/huggingface/datasets/kmewhort___parquet/kmewhort--quickdraw-bins-1pct-sample-eaac784574b05f1b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 2/2 [00:00<00:00,  4.67it/s]


In [4]:
print(dataset)
from struct import unpack
import io

DatasetDict({
    train: Dataset({
        features: ['label', 'packed_drawing'],
        num_rows: 403410
    })
    test: Dataset({
        features: ['label', 'packed_drawing'],
        num_rows: 100853
    })
})


In [5]:
def unpack_drawing(file_handle):
    key_id, = unpack('Q', file_handle.read(8))
    country_code, = unpack('2s', file_handle.read(2))
    recognized, = unpack('b', file_handle.read(1))
    timestamp, = unpack('I', file_handle.read(4))
    n_strokes, = unpack('H', file_handle.read(2))
    image = []
    n_bytes = 17
    for i in range(n_strokes):
        n_points, = unpack('H', file_handle.read(2))
        fmt = str(n_points) + 'B'
        x = unpack(fmt, file_handle.read(n_points))
        y = unpack(fmt, file_handle.read(n_points))
        image.append((x, y))
        n_bytes += 2 + 2*n_points
    result = {
        'key_id': key_id,
        'country_code': country_code,
        'recognized': recognized,
        'timestamp': timestamp,
        'image': image,
    }
    return result

# packed bin -> RGB PIL
def binToPIL(packed_drawing):
    padding = 8
    radius = 7
    scale = (224.0-(2*padding)) / 256
    
    unpacked = unpack_drawing(io.BytesIO(packed_drawing))
    unpacked_image = unpacked['image']
    image = np.full((224,224), 255, np.uint8)
    for stroke in unpacked['image']:
        prevX = round(stroke[0][0]*scale)
        prevY = round(stroke[1][0]*scale)
        for i in range(1, len(stroke[0])):
            x = round(stroke[0][i]*scale)
            y = round(stroke[1][i]*scale)
            cv2.line(image, (padding+prevX, padding+prevY), (padding+x, padding+y), 0, radius, -1)
            prevX = x
            prevY = y
    # turn image into 28 x 28
    image = cv2.resize(image, (28, 28))
    return image
    pilImage = Image.fromarray(image).convert("RGB")     
    return pilImage


# unpack_drawing(dataset['train'][0]['features']['packed_drawing'])
# unpack_drawing(io.BytesIO(dataset['train'][0]['packed_drawing']))

In [6]:
# x_train = np.array([str(dataset['train'][i]['packed_drawing']) for i in range(len(dataset['train']))])
x_temp = np.array([dataset['train'][i]['packed_drawing'] for i in range(len(dataset['train']))])
y_temp = np.array([dataset['train'][i]['label'] for i in range(len(dataset['train']))])

# print(x_temp.shape)

# randomize the data
p = np.random.permutation(len(x_temp))
x_temp = x_temp[p]
y_temp = y_temp[p]

# only use a thousand samples for now
# x_temp = x_temp[:1000]
# y_temp = y_temp[:1000]

print(x_temp.shape)
# testing temp

(403410,)


In [16]:
print(x_temp.shape)

(1000,)


In [29]:
# bin to PIL
# binToPIL(str(x_train[35]))
x = []
y = []
counter = 0
for i in range(len(x_temp)):
    try:
        x.append(binToPIL(x_temp[i]))
        y.append(y_temp[i])
    except:
        counter += 1
        # print(x_temp[i])

print(counter)
# to numpy array
x = np.array(x)
y = np.array(y)

# split into train and test
split = 0.8
x_train = x[:int(split * len(x))]
y_train = y[:int(split * len(y))]
x_test = x[int(split * len(x)) :]
y_test = y[int(split * len(y)) :]

61


In [30]:
image_size = 28

x_train = x_train.reshape(x_train.shape[0], image_size, image_size, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], image_size, image_size, 1).astype('float32')

x_train /= 255.0
x_test /= 255.0

In [31]:
# on the fly byte tensor to np array layer
class ByteTensorToNumpyArray(layers.Layer):
    def __init__(self, **kwargs):
        super(ByteTensorToNumpyArray, self).__init__(**kwargs)

    def call(self, inputs):
        return tf.numpy_function(func=binToPIL, inp=[inputs], Tout=tf.uint8)
    
layer = ByteTensorToNumpyArray()


In [32]:
# Convert class vectors to class matrices
y_train = keras.utils.to_categorical(y_train, 345)
y_test = keras.utils.to_categorical(y_test, 345)

model = keras.Sequential()
model.add(layers.Convolution2D(16, (3, 3),
                        padding='same',
                        input_shape=x_train.shape[1:], activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Convolution2D(32, (3, 3), padding='same', activation= 'relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Convolution2D(64, (3, 3), padding='same', activation= 'relu'))
model.add(layers.MaxPooling2D(pool_size =(2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(345, activation='softmax')) 
# Train model
# adam = tf.train.AdamOptimizer()
# Train model
# adam = tf.train.AdamOptimizer()
# model.compile(loss='categorical_crossentropy',
#               optimizer=adam,
#               metrics=['top_k_categorical_accuracy'])

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['top_k_categorical_accuracy'])

print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 28, 28, 16)        160       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 14, 14, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 14, 14, 32)        4640      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 7, 7, 32)         0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 7, 7, 64)          18496     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 3, 3, 64)        

In [33]:
model.fit(x = x_train, y = y_train, validation_split=0.1, batch_size = 256, verbose=2, epochs=5)

#evaluate on unseen data
score = model.evaluate(x_test, y_test, verbose=0)
print('Test accuarcy: {:0.2f}%'.format(score[1] * 100))

Epoch 1/5
3/3 - 3s - loss: 5.8442 - top_k_categorical_accuracy: 0.0089 - val_loss: 5.8555 - val_top_k_categorical_accuracy: 0.0132 - 3s/epoch - 841ms/step
Epoch 2/5
3/3 - 0s - loss: 5.8224 - top_k_categorical_accuracy: 0.0207 - val_loss: 5.8609 - val_top_k_categorical_accuracy: 0.0132 - 403ms/epoch - 134ms/step
Epoch 3/5
3/3 - 1s - loss: 5.7949 - top_k_categorical_accuracy: 0.0267 - val_loss: 5.8856 - val_top_k_categorical_accuracy: 0.0132 - 557ms/epoch - 186ms/step
Epoch 4/5
3/3 - 0s - loss: 5.7512 - top_k_categorical_accuracy: 0.0415 - val_loss: 5.9658 - val_top_k_categorical_accuracy: 0.0132 - 485ms/epoch - 162ms/step
Epoch 5/5
3/3 - 0s - loss: 5.7095 - top_k_categorical_accuracy: 0.0415 - val_loss: 6.0559 - val_top_k_categorical_accuracy: 0.0132 - 377ms/epoch - 126ms/step
Test accuarcy: 1.06%


In [45]:
import numpy as np
# get the size of the first element in the training set
# this is the size of the image
image_size = x_train[0].shape[0]
print(image_size)

# create an array of zeros with the same size as the image
# this will be the image that we will draw on
image = np.zeros((image_size, image_size))
# reshape the image to be a fit for the model
image = image.reshape(1, image_size, image_size, 1).astype('float32')

28


In [46]:
# save the image as a json file
import json
with open('image.json', 'w') as outfile:
    json.dump(image.tolist(), outfile)

In [38]:
# predict using the first image in the test set
print(model.predict(x_test[0:1]))

1/1 [==============================] - 1s 1s/step
[[0.002131   0.00465648 0.00196245 0.00271681 0.00276198 0.00118648
  0.00049879 0.00037218 0.00156709 0.00476362 0.00212396 0.00064234
  0.00256845 0.00145623 0.00285067 0.001548   0.00192683 0.00209718
  0.00175823 0.00635993 0.00224878 0.00193923 0.00081721 0.00338876
  0.00072713 0.00147774 0.00226917 0.00367905 0.00197505 0.00090464
  0.0026802  0.00369431 0.00152704 0.00929076 0.0019763  0.00114994
  0.00158516 0.0018165  0.00071738 0.00179795 0.00685635 0.00154472
  0.00549464 0.00083173 0.00094284 0.00118036 0.0005366  0.00200062
  0.0050106  0.00069132 0.00494606 0.00412925 0.0021354  0.00278046
  0.00054942 0.00147162 0.00716976 0.00086851 0.00146297 0.00269359
  0.00676326 0.01109182 0.00294942 0.00441382 0.00419435 0.0017698
  0.00246963 0.00186172 0.00116578 0.00147779 0.00237436 0.00560486
  0.00270869 0.00497384 0.00267395 0.00263405 0.00654821 0.00204477
  0.00307611 0.00214549 0.00254805 0.00068134 0.001637   0.00313682

In [34]:
model.save('model.h5')

In [37]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)